# This notebook is intended to create the initial knowledgebase for pinecone. We shall be having another page for expanding the knowledgebase


In [6]:
import pandas as pd


In [7]:

df = pd.read_excel("Knowledgebase.xlsx")

In [8]:
df.columns

Index(['ITEM Description', 'GOEDEREN OMSCHRIJVING', 'Goederen Code',
       'CATEGORY'],
      dtype='object')

In [9]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [10]:
df

,ITEM Description,GOEDEREN OMSCHRIJVING,Goederen Code,CATEGORY
0,Chemical Guys AIR_301_04 Sample Kit,LUCHTVERFRISSER,33073000,NaN
1,LITTLE TREES Car Air Freshener (Cherry),LUCHTVERFRISSER,33073000,NaN
2,LITTLE TREES Car Air Freshener (Watermelon),LUCHTVERFRISSER,33073000,NaN
3,Peach Berry Car Air Fresheners,LUCHTVERFRISSER,33073000,NaN
4,Air Jungles New Car Scent Clip,LUCHTVERFRISSER,33073000,NaN
...,...,...,...,...
138,Elenzga New Women's Midi Dress,DAMES KLEDING,61042200,NaN
139,Mi Lightweight Zipper Hobo Bag,TASSEN,42029900,NaN
140,Women's Square Toe Stiletto High Heel Sandals,DAMES SCHOENEN,64051000,NaN
141,3pcs Women's Golden Styling Buckle Belts,RIEMEN,42033000,NaN


In [ ]:
items = df["ITEM Description"]
names = df["GOEDEREN OMSCHRIJVING"]
codes = df["Goederen Code"]
categories = df["CATEGORY"]

In [12]:
df.iterrows()

<generator object DataFrame.iterrows at 0x000001FFFA17B9A0>

In [15]:
for index, row in df.iterrows():
  print(row)

ITEM Description         Chemical Guys AIR_301_04 Sample Kit
GOEDEREN OMSCHRIJVING                        LUCHTVERFRISSER
Goederen Code                                       33073000
CATEGORY                                                 NaN
Name: 0, dtype: object
ITEM Description         LITTLE TREES Car Air Freshener (Cherry)
GOEDEREN OMSCHRIJVING                            LUCHTVERFRISSER
Goederen Code                                           33073000
CATEGORY                                                     NaN
Name: 1, dtype: object
ITEM Description         LITTLE TREES Car Air Freshener (Watermelon)
GOEDEREN OMSCHRIJVING                                LUCHTVERFRISSER
Goederen Code                                               33073000
CATEGORY                                                         NaN
Name: 2, dtype: object
ITEM Description         Peach Berry Car Air Fresheners
GOEDEREN OMSCHRIJVING                   LUCHTVERFRISSER
Goederen Code                          

In [2]:
from langchain.embeddings.openai import OpenAIEmbeddings
from pinecone import Pinecone
from pinecone import ServerlessSpec
from langchain_pinecone import PineconeVectorStore

c:\Users\LENOVO\Desktop\projects-only\projects-only\werkbrief-generator\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\LENOVO\Desktop\projects-only\projects-only\werkbrief-generator\.venv\Lib\site-packages\langchain_pinecone\__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecon

In [ ]:
embeddings = OpenAIEmbeddings()  

In [5]:
import getpass
import os
# Pinecone client
if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

In [ ]:
index_name = "ship2aruba" 
if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

In [ ]:
index = pc.Index(index_name)
vector_store = PineconeVectorStore(index=index, embedding=embeddings)


In [16]:
from langchain_core.documents import Document
# How pinecone expects each chunk to be
docs = [Document(page_content="The description of this item is:"+ row['ITEM Description'] + "/nThe description of the goods is:"+row["GOEDEREN OMSCHRIJVING"],metadata={"desc":row["ITEM Description"],"code":row["Goederen Code"],"gdesc":row["GOEDEREN OMSCHRIJVING"],"category":row["CATEGORY"]}) for index, row in df.iterrows()]


In [17]:
docs

[Document(metadata={'desc': 'Chemical Guys AIR_301_04 Sample Kit', 'code': 33073000, 'gdesc': 'LUCHTVERFRISSER', 'category': nan}, page_content='The description of this item is:Chemical Guys AIR_301_04 Sample Kit/nThe description of the goods is:LUCHTVERFRISSER'),
 Document(metadata={'desc': 'LITTLE TREES Car Air Freshener (Cherry)', 'code': 33073000, 'gdesc': 'LUCHTVERFRISSER', 'category': nan}, page_content='The description of this item is:LITTLE TREES Car Air Freshener (Cherry)/nThe description of the goods is:LUCHTVERFRISSER'),
 Document(metadata={'desc': 'LITTLE TREES Car Air Freshener (Watermelon)', 'code': 33073000, 'gdesc': 'LUCHTVERFRISSER', 'category': nan}, page_content='The description of this item is:LITTLE TREES Car Air Freshener (Watermelon)/nThe description of the goods is:LUCHTVERFRISSER'),
 Document(metadata={'desc': 'Peach Berry Car Air Fresheners', 'code': 33073000, 'gdesc': 'LUCHTVERFRISSER', 'category': nan}, page_content='The description of this item is:Peach Ber

In [19]:
# Now add all of the docs in the pinceone namespace
uuids = [str(doc.metadata["code"]) for doc in ((docs))]
batch_size = 10
for i in range(0, len(docs), batch_size):
    batch = docs[i:i+batch_size]
    batch_ids = uuids[i:i+batch_size]
    print("Current Batch Index is:",batch_ids)
    # vector_store.add_documents(batch,ids=batch_ids)

Current Batch Index is: ['33073000', '33073000', '33073000', '33073000', '33073000', '33073000', '33073000', '33073000', '39249090', '95051000']
Current Batch Index is: ['85094000', '94054050', '84733000', '85078000', '82141000', '94054050', '64051000', '96132090', '84818000', '85238000']
Current Batch Index is: ['33049900', '33049900', '33049900', '33049900', '33049900', '33049900', '33049900', '33049900', '33049900', '33049900']
Current Batch Index is: ['33049900', '33049900', '39269090', '95059000', '71171910', '82142000', '33049900', '30051000', '61042200', '71171910']
Current Batch Index is: ['95059000', '95059000', '82142000', '71171910', '49119100', '71171910', '61042200', '61082100', '59061000', '95059000']
Current Batch Index is: ['34011900', '85131000', '42010000', '61124100', '61042200', '61042200', '61124100', '61042200', '61042200', '61042200']
Current Batch Index is: ['61042200', '61091000', '61042200', '61042200', '61124100', '61042200', '61091000', '61042200', '61124100